# Pixel Drill for Indices outputs to CSV - Multi-site

| Authors:  | Bex Dunn|
|----------|----------------|
| Created: | March 6, 2019 |
| Last edited: | March 6, 2019 |

**Requirements:**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

__Background:__ Data from the [Landsat](https://landsat.usgs.gov/about-landsat) 5,7 and 8 satellite missions are accessible through [Digital Earth Australia](http://www.ga.gov.au/about/projects/geographic/digital-earth-australia) (DEA).

__What does this notebook do?:__ This notebook takes a supplied CSV of site points. It runs a pixel drill through surface reflectance, calculates NDVI, Taselled cap wetness and greenness, and outputs a csv of values for each site and plots of each index for each site.

**Tags**: :index:`Landsat`,:index:`Landsat5`,:index:`Landsat7`,:index:`Landsat8`, :index:`pixeldrill`, :index:`DEAPlotting`, :index:`datacube.utils.geometry`, :index:`query`,:index:`Scripts`,:index:`tasseled_cap`, :index:`NDVI`,                                                                                                           :index:`DEADataHandling`, :index:`DEAPlotting`, :index:`load_clearlandsat`

import some modules

In [5]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import shapely
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, BandIndices

dc = datacube.Datacube(app='pixel drill')

%load_ext autoreload

%autoreload 2

#set up file to open 

inpath = '/g/data/r78/rjd547/LTIM_veg_monitoring/LTIM vegetation monitoring sites.csv'

### Convert csv latitude and longitude values into a geopandas geodatafrome

veg_sites = pd.read_csv(inpath, delimiter=",")
#turn csv geometry into geopandas dataframe geometry using lambda functions and shapely
veg_sites['geometry']=veg_sites.apply(lambda z: shapely.geometry.Point(z.LATITUDE, z.LONGITUDE), axis=1)
veg_sites = gpd.GeoDataFrame(veg_sites)

#define an output location
output_loc = '/g/data/r78/rjd547/LTIM_veg_monitoring/'

### run this for multiple sites

for site in range(0,len(veg_sites)):
    print(f'running for index {site}')

    query = {'lat':(veg_sites['LATITUDE'][site]), 
             'lon':(veg_sites['LONGITUDE'][site])}               

    Veg_Site = '_'.join(veg_sites['NAME'][site].split(' '))
    print(Veg_Site)

    outfilename=output_loc+Veg_Site+'.csv'
    print (outfilename)

    ls578 = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart')

    ### Calculate NDVI 

    ### Calculate the tasseled cap indices

    #transform the nbart into tci - we want unthresholded, which is why we set drop_tci_bands to False
    tci = TasseledCapTools.thresholded_tasseled_cap(ls578,wetness_threshold=-350, drop=True , drop_tc_bands=False)

    tcw = tci.wetness
    tcg = tci.greenness

    ndvi = BandIndices.calculate_indices(ls578, index='NDVI')

    ### create and fill pandas dataframe to write to csv

    #drop dimensions of length 1
    ndvi = ndvi.squeeze('x')
    ndvi = ndvi.squeeze('y')

    #make a new dataframe using the data from the xarray of ndvi
    NDVI = pd.DataFrame(data=ndvi.data, index=ndvi.time,columns=['ndvi'])

    #rename the dataframe so that we can add the other indexes
    INDICIES=NDVI

    #again drop dims
    TCW = tcw.squeeze().data
    TCG = tcg.squeeze().data

    # set up final dataframe to write to file 'INDICES'

    INDICIES['tcw']=TCW
    INDICIES['tcg']=TCG
    #we have some values where the pixel is cloudy. Drop these values
    INDICES = INDICIES.dropna(axis='index', thresh =2)

    INDICES.to_csv(outfilename)

    !ls /g/data/r78/rjd547/LTIM_veg_monitoring/*.csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
running for index 0
GWY_MUNG1
/g/data/r78/rjd547/LTIM_veg_monitoring/GWY_MUNG1.csv
Loading ls5
    Loading 338 filtered ls5 timesteps
Loading ls7
    Ignoring SLC-off observations for ls7
    Loading 73 filtered ls7 timesteps
Loading ls8
    Loading 131 filtered ls8 timesteps
Combining and sorting ls5, ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)
The formula we are using is (nir - red)/(nir + red)
/g/data/r78/rjd547/LTIM_veg_monitoring/GWY_MUNG1.csv
/g/data/r78/rjd547/LTIM_veg_monitoring/LTIM vegetation monitoring sites.csv
/g/data/r78/rjd547/LTIM_veg_monitoring/MDBA_SCA_Tool_uniall_scores.csv
running for index 1
GWY_ODB1
/g/data/r78/rjd547/LTIM_veg_monitoring/GWY_ODB1.csv
Loading ls5
    Loading 338 filtered ls5 timesteps
Loading ls7
    Ignoring SLC-off observations for ls7
    Loading 73 filtered ls7 timesteps
Loading ls8
    Loading 131 filtered ls8 

AttributeError: 'float' object has no attribute 'split'